In [3]:
# Download all countries data from World Bank
def get_all_countries_population():
    """
    Get population data for all countries to calculate ROW regions
    """
    # This gets all countries
    url = "https://api.worldbank.org/v2/country/all/indicator/SP.POP.TOTL?date=2022&format=json&per_page=300"
    
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        
        # Extract country data
        countries = []
        if len(data) > 1:
            for item in data[1]:
                if item['value']:
                    countries.append({
                        'country_code': item['country']['id'],
                        'country_name': item['country']['value'],
                        'population_2022': item['value']
                    })
        
        return pd.DataFrame(countries)
    return None

# Get all countries
all_countries = get_all_countries_population()

# Save for ROW calculations
all_countries.to_csv('all_countries_population_2022.csv', index=False)

In [2]:
import pandas as pd
import requests
import time
import numpy as np

# World Bank API base URL
WB_API_BASE = "https://api.worldbank.org/v2/country/{}/indicator/SP.POP.TOTL?date=2022&format=json&per_page=1"

# ISO2 to ISO3 mapping (standard codes)
iso2_to_iso3 = {
    'AF': 'AFG', 'AL': 'ALB', 'DZ': 'DZA', 'AS': 'ASM', 'AD': 'AND', 'AO': 'AGO',
    'AI': 'AIA', 'AQ': 'ATA', 'AG': 'ATG', 'AR': 'ARG', 'AM': 'ARM', 'AW': 'ABW',
    'AZ': 'AZE', 'BS': 'BHS', 'BH': 'BHR', 'BD': 'BGD', 'BB': 'BRB', 'BY': 'BLR',
    'BZ': 'BLZ', 'BJ': 'BEN', 'BM': 'BMU', 'BT': 'BTN', 'BO': 'BOL', 'BA': 'BIH',
    'BW': 'BWA', 'BN': 'BRN', 'BF': 'BFA', 'BI': 'BDI', 'KH': 'KHM', 'CM': 'CMR',
    'CV': 'CPV', 'KY': 'CYM', 'CF': 'CAF', 'TD': 'TCD', 'CL': 'CHL', 'CO': 'COL',
    'KM': 'COM', 'CD': 'COD', 'CG': 'COG', 'CK': 'COK', 'CR': 'CRI', 'CI': 'CIV',
    'CU': 'CUB', 'DJ': 'DJI', 'DM': 'DMA', 'DO': 'DOM', 'EC': 'ECU', 'EG': 'EGY',
    'SV': 'SLV', 'GQ': 'GNQ', 'ER': 'ERI', 'ET': 'ETH', 'FO': 'FRO', 'FK': 'FLK',
    'FJ': 'FJI', 'GF': 'GUF', 'PF': 'PYF', 'GA': 'GAB', 'GM': 'GMB', 'GE': 'GEO',
    'GH': 'GHA', 'GI': 'GIB', 'GL': 'GRL', 'GD': 'GRD', 'GP': 'GLP', 'GU': 'GUM',
    'GT': 'GTM', 'GN': 'GIN', 'GW': 'GNB', 'GY': 'GUY', 'HT': 'HTI', 'HN': 'HND',
    'HK': 'HKG', 'IS': 'ISL', 'IR': 'IRN', 'IQ': 'IRQ', 'IM': 'IMN', 'IL': 'ISR',
    'JM': 'JAM', 'JO': 'JOR', 'KZ': 'KAZ', 'KE': 'KEN', 'KI': 'KIR', 'KP': 'PRK',
    'KW': 'KWT', 'KG': 'KGZ', 'LA': 'LAO', 'LB': 'LBN', 'LS': 'LSO', 'LR': 'LBR',
    'LY': 'LBY', 'LI': 'LIE', 'MO': 'MAC', 'MK': 'MKD', 'MG': 'MDG', 'MW': 'MWI',
    'MY': 'MYS', 'MV': 'MDV', 'ML': 'MLI', 'MH': 'MHL', 'MQ': 'MTQ', 'MR': 'MRT',
    'MU': 'MUS', 'YT': 'MYT', 'FM': 'FSM', 'MD': 'MDA', 'MC': 'MCO', 'MN': 'MNG',
    'ME': 'MNE', 'MS': 'MSR', 'MA': 'MAR', 'MZ': 'MOZ', 'MM': 'MMR', 'NA': 'NAM',
    'NR': 'NRU', 'NP': 'NPL', 'AN': 'ANT', 'NC': 'NCL', 'NZ': 'NZL', 'NI': 'NIC',
    'NE': 'NER', 'NG': 'NGA', 'NU': 'NIU', 'MP': 'MNP', 'OM': 'OMN', 'PK': 'PAK',
    'PW': 'PLW', 'PS': 'PSE', 'PA': 'PAN', 'PG': 'PNG', 'PY': 'PRY', 'PE': 'PER',
    'PH': 'PHL', 'PR': 'PRI', 'QA': 'QAT', 'RE': 'REU', 'RW': 'RWA', 'WS': 'WSM',
    'SM': 'SMR', 'ST': 'STP', 'SA': 'SAU', 'SN': 'SEN', 'RS': 'SRB', 'SC': 'SYC',
    'SL': 'SLE', 'SG': 'SGP', 'SB': 'SLB', 'SO': 'SOM', 'LK': 'LKA', 'KN': 'KNA',
    'LC': 'LCA', 'PM': 'SPM', 'VC': 'VCT', 'SD': 'SDN', 'SR': 'SUR', 'SZ': 'SWZ',
    'SY': 'SYR', 'TJ': 'TJK', 'TZ': 'TZA', 'TH': 'THA', 'TP': 'TLS', 'TG': 'TGO',
    'TO': 'TON', 'TT': 'TTO', 'TN': 'TUN', 'TM': 'TKM', 'TC': 'TCA', 'TV': 'TUV',
    'UG': 'UGA', 'UA': 'UKR', 'AE': 'ARE', 'UY': 'URY', 'UZ': 'UZB', 'VU': 'VUT',
    'VA': 'VAT', 'VE': 'VEN', 'VN': 'VNM', 'VG': 'VGB', 'VI': 'VIR', 'EH': 'ESH',
    'YE': 'YEM', 'ZM': 'ZMB', 'ZW': 'ZWE', 'XK': 'XKX'  # Kosovo with World Bank code
}

# Update: For Kosovo, check both XK and special "Kosovo" string
# Note: Gibraltar already has GIB in the mapping above

# ROW region mapping from your Excel file
row_countries = {
    'WA': [  # Rest of Asia and Pacific
        'AF', 'AS', 'AQ', 'AM', 'AZ', 'BD', 'BT', 'BV', 'B1', 'IO', 'BN', 'KH',
        'CX', 'CC', 'CK', 'FJ', 'PF', 'GE', 'GU', 'HM', 'HK', 'KZ', 'KI', 'KP',
        'KG', 'LA', 'MO', 'MY', 'MV', 'MH', 'FM', 'MN', 'MM', 'NR', 'NP', 'NC',
        'NZ', 'NU', 'NF', 'MP', 'PK', 'PW', 'PG', 'PH', 'PN', 'WS', 'SG', 'SB',
        'GS', 'LK', 'TJ', 'TH', 'TP', 'TK', 'TO', 'TM', 'TV', 'UM', 'UZ', 'VU',
        'VN', 'WF'
    ],
    'WL': [  # Rest of Latin America
        'AI', 'AG', 'AR', 'AW', 'BS', 'BB', 'BZ', 'BM', 'BO', 'KY', 'CL', 'CO',
        'CR', 'CU', 'DM', 'DO', 'EC', 'SV', 'FK', 'GL', 'GD', 'GP', 'GT', 'GY',
        'HT', 'HN', 'JM', 'MQ', 'AN', 'NI', 'PA', 'PY', 'PE', 'PR', 'KN', 'LC',
        'PM', 'VC', 'SR', 'TT', 'TC', 'UY', 'VE', 'VG', 'VI', 'GF'
    ],
    'WE': [  # Rest of Europe  
        'AL', 'AD', 'BY', 'BA', 'Channel Islands', 'FO', 'GI', 'IS', 'IM', 'Kosovo',
        'LI', 'MK', 'MD', 'MC', 'ME', 'SM', 'RS', 'SJ', 'UA', 'VA'
    ],
    'WF': [  # Rest of Africa
        'DZ', 'AO', 'BJ', 'BW', 'BF', 'BI', 'CM', 'CV', 'CF', 'TD', 'KM', 'CD',
        'CG', 'CI', 'DJ', 'GQ', 'ER', 'ET', 'GA', 'GM', 'GH', 'GN',
        'GW', 'KE', 'LS', 'LR', 'LY', 'MG', 'MW', 'ML', 'MR', 'MU', 'YT', 'MA',
        'MZ', 'NA', 'NE', 'NG', 'RE', 'RW', 'ST', 'SN', 'SC', 'SL', 'SO', 'SH',
        'SD', 'SZ', 'TZ', 'TG', 'TN', 'UG', 'EH', 'ZM', 'ZW'
    ],
    'WM': [  # Rest of Middle East
        'BH', 'IR', 'IQ', 'IL', 'JO', 'KW', 'LB', 'OM', 'PS', 'QA', 'SA',
        'SY', 'AE', 'YE','EG'
    ]
}

def get_population_worldbank(country_code, year=2022):
    """Get population from World Bank API"""
    url = WB_API_BASE.format(country_code)
    
    try:
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            if len(data) > 1 and data[1] and data[1][0]['value']:
                return data[1][0]['value'], "World Bank (2022)"
    except:
        pass
    return None, None

# Additional official sources for territories not in World Bank
official_territory_data = {
    # From UN Data or official government statistics
    'TW': (23894394, "Taiwan National Statistics (2022)"),
    'CK': (17565, "Cook Islands Statistics Office (2022)"),
    'NU': (1700, "Niue Statistics Office (2022)"),
    'VA': (825, "Vatican City State Statistics (2022)"),
    'AI': (15857, "Anguilla Statistics Department (2022)"),
    'FK': (3662, "Falkland Islands Government Census (2021)"),
    'IM': (84069, "Isle of Man Government (2022)"),
    'JE': (103267, "Jersey Statistics Unit (2022)"),
    'GG': (63026, "Guernsey Statistics (2022)"),
    'SH': (4439, "St Helena Government (2021 Census)"),
    'PN': (47, "Pitcairn Islands Government (2022)"),
    'WF': (11558, "Wallis and Futuna INSEE (2018 Census)"),
    'PM': (6008, "St Pierre and Miquelon INSEE (2022)"),
    'GP': (395700, "Guadeloupe INSEE (2022)"),
    'MQ': (361225, "Martinique INSEE (2022)"),
    'GF': (294436, "French Guiana INSEE (2022)"),
    'YT': (310022, "Mayotte INSEE (2022)"),
    'RE': (873102, "Réunion INSEE (2022)"),
    'EH': (563202, "Westsahara Bevölkerung (2025)"),  # Western Sahara
    # Uninhabited or research stations
    'AQ': (0, "Antarctica - No permanent population"),
    'BV': (0, "Bouvet Island - Uninhabited"),
    'HM': (0, "Heard and McDonald Islands - Uninhabited"),
    'GS': (0, "South Georgia - No permanent population"),
    'IO': (0, "British Indian Ocean Territory - No permanent population"),
    'UM': (0, "US Minor Outlying Islands - No permanent population"),
}

# Collect population data
all_population_data = []
missing_data = []

print("Fetching population data from official sources...\n")

# First, get data for individual EXIOBASE3 countries
exiobase3_individual = ['AT', 'BE', 'BG', 'CY', 'CZ', 'DE', 'DK', 'EE', 'ES', 'FI', 
                       'FR', 'GR', 'HR', 'HU', 'IE', 'IT', 'LT', 'LU', 'LV', 'MT', 
                       'NL', 'PL', 'PT', 'RO', 'SE', 'SI', 'SK', 'GB', 'US', 'JP', 
                       'CN', 'CA', 'KR', 'BR', 'IN', 'MX', 'RU', 'AU', 'CH', 'TR', 
                       'TW', 'NO', 'ID', 'ZA']

print("Individual EXIOBASE3 Countries:")
for iso2 in exiobase3_individual:
    if iso2 == 'TW':  # Taiwan special case
        pop, source = official_territory_data['TW']
    else:
        iso3 = iso2_to_iso3.get(iso2, iso2)
        pop, source = get_population_worldbank(iso3)
    
    if pop:
        all_population_data.append({
            'EXIOBASE3_Code': iso2,
            'ISO2': iso2,
            'Population_2022': pop,
            'Source': source,
            'Type': 'Individual Country'
        })
        print(f"{iso2}: {pop:,.0f} - {source}")
    time.sleep(0.05)

# Now process ROW regions
print("\n\nRest of World Regions:")
row_totals = {}

for region, countries in row_countries.items():
    print(f"\n{region}:")
    region_total = 0
    region_data = []
    
    for iso2 in countries:
        pop = None
        source = None
        
        # Special handling for Kosovo
        if iso2 == 'Kosovo':
            # Try World Bank with XKX code
            pop, source = get_population_worldbank('XKX')
            if not pop:
                print(f"  Kosovo: Trying alternative World Bank lookup...")
        # Check official territory data
        elif iso2 in official_territory_data:
            pop, source = official_territory_data[iso2]
        # Special handling for Channel Islands
        elif iso2 == 'Channel Islands':
            # Sum Jersey and Guernsey
            jersey_pop, _ = official_territory_data.get('JE', (0, ""))
            guernsey_pop, _ = official_territory_data.get('GG', (0, ""))
            pop = jersey_pop + guernsey_pop
            source = "Jersey + Guernsey Statistics (2022)"
        # Skip codes that aren't real territories
        elif iso2 in ['B1', 'CX', 'CC', 'NF', 'TK', 'SJ', 'AN']:
            print(f"  {iso2}: Skipped - Not a standard territory code")
            continue
        else:
            # Try World Bank
            iso3 = iso2_to_iso3.get(iso2)
            if iso3:
                pop, source = get_population_worldbank(iso3)
        
        if pop is not None:
            region_total += pop
            region_data.append({
                'ISO2': iso2,
                'ROW_Region': region,
                'Population_2022': pop,
                'Source': source
            })
            all_population_data.append({
                'EXIOBASE3_Code': region,
                'ISO2': iso2,
                'Population_2022': pop,
                'Source': source,
                'Type': f'Part of {region}'
            })
            if pop > 0:
                print(f"  {iso2}: {pop:,.0f} - {source}")
        else:
            missing_data.append({'ISO2': iso2, 'Region': region})
            print(f"  {iso2}: NO DATA FOUND")
        
        time.sleep(0.05)
    
    row_totals[region] = region_total
    all_population_data.append({
        'EXIOBASE3_Code': region,
        'ISO2': region,
        'Population_2022': region_total,
        'Source': 'Sum of constituent countries',
        'Type': 'ROW Total'
    })
    print(f"\n{region} TOTAL: {region_total:,.0f}")

# Create final summary for EXIOBASE3
exiobase3_summary = []

# Add individual countries
for iso2 in exiobase3_individual:
    pop_data = next((d for d in all_population_data if d['EXIOBASE3_Code'] == iso2 and d['Type'] == 'Individual Country'), None)
    if pop_data:
        exiobase3_summary.append({
            'EXIOBASE3_Code': iso2,
            'Population_2022': pop_data['Population_2022'],
            'Source': pop_data['Source']
        })

# Add ROW totals
for region in ['WA', 'WL', 'WE', 'WF', 'WM']:
    exiobase3_summary.append({
        'EXIOBASE3_Code': region,
        'Population_2022': row_totals[region],
        'Source': 'Sum of constituent countries (see details file)'
    })

# Convert to DataFrames and save
df_summary = pd.DataFrame(exiobase3_summary)
df_all_details = pd.DataFrame(all_population_data)
df_missing = pd.DataFrame(missing_data)

# Calculate total
total_pop = df_summary['Population_2022'].sum()
print(f"\n\nTotal EXIOBASE3 Population: {total_pop:,.0f}")
print(f"This equals approximately {total_pop/1e9:.2f} billion people")

# Save files
df_summary.to_csv('exiobase3_population_2022_official.csv', index=False)
df_all_details.to_csv('population_all_details_with_sources_2022.csv', index=False)
df_missing.to_csv('missing_population_data.csv', index=False)

print("\n\nFiles saved:")
print("1. exiobase3_population_2022_official.csv - Main file for your analysis")
print("2. population_all_details_with_sources_2022.csv - All data with sources")
print("3. missing_population_data.csv - Territories without data")

Fetching population data from official sources...

Individual EXIOBASE3 Countries:
AT: 9,041,851 - World Bank (2022)
BE: 11,680,210 - World Bank (2022)
BG: 6,643,324 - World Bank (2022)
CY: 1,331,370 - World Bank (2022)
CZ: 10,672,118 - World Bank (2022)
DE: 83,797,985 - World Bank (2022)
DK: 5,903,037 - World Bank (2022)
EE: 1,348,840 - World Bank (2022)
ES: 47,759,127 - World Bank (2022)
FI: 5,556,106 - World Bank (2022)
FR: 68,065,015 - World Bank (2022)
GR: 10,436,882 - World Bank (2022)
HR: 3,855,641 - World Bank (2022)
HU: 9,644,377 - World Bank (2022)
IE: 5,165,700 - World Bank (2022)
IT: 59,013,667 - World Bank (2022)
LT: 2,831,639 - World Bank (2022)
LU: 653,103 - World Bank (2022)
LV: 1,879,383 - World Bank (2022)
MT: 531,511 - World Bank (2022)
NL: 17,700,982 - World Bank (2022)
PL: 36,821,749 - World Bank (2022)
PT: 10,434,332 - World Bank (2022)
RO: 19,048,502 - World Bank (2022)
SE: 10,486,941 - World Bank (2022)
SI: 2,112,076 - World Bank (2022)
SK: 5,431,752 - World Ban